In [ ]:
import pandas as pd
from datasets import Dataset
from sklearn.preprocessing import LabelEncoder

df = pd.read_csv("../data/processed/Laptop_Train_Cleaned.csv")

label_encoder = LabelEncoder()
df["label"] = label_encoder.fit_transform(df["polarity"]) 

dataset = Dataset.from_pandas(df)
print(dataset)


c:\Users\USER\OneDrive\Bureau\FTM-absa-product-review\absa_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Dataset({
    features: ['id', 'Sentence', 'Aspect Term', 'polarity', 'from', 'to', 'label'],
    num_rows: 2358
})


In [26]:
df["Sentence"][0]

'I charge it at night and skip taking the cord with me because of the good battery life.'

In [11]:
from transformers import BertTokenizerFast

tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")

def tokenize_function(example):
    return tokenizer(
        example["Sentence"],
        example["Aspect Term"],
        truncation=True,
        padding="max_length",
        max_length=128
    )

tokenized_dataset = dataset.map(tokenize_function, batched=True)
tokenized_dataset = tokenized_dataset.rename_column("label", "labels")
tokenized_dataset.set_format("torch")


Map: 100%|██████████| 2358/2358 [00:00<00:00, 4185.03 examples/s]


In [18]:
from transformers import BertForSequenceClassification

num_labels = len(label_encoder.classes_)
asc_model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=num_labels
)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="../models/asc_model",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    save_strategy="epoch",
    logging_dir="./logs",
    learning_rate=2e-5,
    weight_decay=0.01
)

def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    return {
        "accuracy": accuracy_score(p.label_ids, preds),
        "precision": precision_score(p.label_ids, preds, average="weighted"),
        "recall": recall_score(p.label_ids, preds, average="weighted"),
        "f1": f1_score(p.label_ids, preds, average="weighted"),
    }

trainer = Trainer(
    model=asc_model,
    args=training_args,
    train_dataset=tokenized_dataset,
    eval_dataset=tokenized_dataset,
    compute_metrics=compute_metrics
)


for epoch in range(int(training_args.num_train_epochs)):
    trainer.train()
    metrics = trainer.evaluate()  
    print(f"Epoch {epoch+1} metrics:", metrics)



c:\Users\USER\OneDrive\Bureau\FTM-absa-product-review\absa_env\Lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss
500,0.260100


c:\Users\USER\OneDrive\Bureau\FTM-absa-product-review\absa_env\Lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
c:\Users\USER\OneDrive\Bureau\FTM-absa-product-review\absa_env\Lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
c:\Users\USER\OneDrive\Bureau\FTM-absa-product-review\absa_env\Lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch 1 metrics: {'eval_loss': 0.08669554442167282, 'eval_accuracy': 0.9800678541136556, 'eval_precision': 0.9802580660145611, 'eval_recall': 0.9800678541136556, 'eval_f1': 0.9796688872766477, 'eval_runtime': 766.0175, 'eval_samples_per_second': 3.078, 'eval_steps_per_second': 0.385, 'epoch': 3.0}


c:\Users\USER\OneDrive\Bureau\FTM-absa-product-review\absa_env\Lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


...

Epoch 2 metrics: {'eval_loss': 0.050802044570446014, 'eval_accuracy': 0.9872773536895675, 'eval_precision': 0.9871947476437528, 'eval_recall': 0.9872773536895675, 'eval_f1': 0.9872143768826219, 'eval_runtime': 422.6463, 'eval_samples_per_second': 5.579, 'eval_steps_per_second': 0.698, 'epoch': 3.0}


c:\Users\USER\OneDrive\Bureau\FTM-absa-product-review\absa_env\Lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss
500,0.118100


c:\Users\USER\OneDrive\Bureau\FTM-absa-product-review\absa_env\Lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
c:\Users\USER\OneDrive\Bureau\FTM-absa-product-review\absa_env\Lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
c:\Users\USER\OneDrive\Bureau\FTM-absa-product-review\absa_env\Lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch 3 metrics: {'eval_loss': 0.029470758512616158, 'eval_accuracy': 0.9919423240033927, 'eval_precision': 0.9919353286798487, 'eval_recall': 0.9919423240033927, 'eval_f1': 0.9919264499284574, 'eval_runtime': 428.5984, 'eval_samples_per_second': 5.502, 'eval_steps_per_second': 0.688, 'epoch': 3.0}
